In [10]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Boolean, Float, Text, SmallInteger
from sqlalchemy.orm import declarative_base
import pathlib

# 1. 파일 경로 설정
PARQUET_FILE_PATH = pathlib.Path("./data/parsed_logs.parquet")

# 2. 데이터베이스 연결 정보 설정
DB_USER = "evan"
DB_PASSWORD = "123456"
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_NAME = "de_etl_book"

# 3. SQLAlchemy 엔진 생성 및 연결 테스트
try:
    engine = create_engine(f'mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
    with engine.connect() as conn:
        print("MySQL 데이터베이스 연결 성공!")
except Exception as e:
    print(f"MySQL 데이터베이스 연결 실패: {e}")
    exit()

# 4. SQLAlchemy ORM 모델 정의 (DataFrame 열 정보에 맞게 재정의)
Base = declarative_base()

class ParsedLogs(Base):
    __tablename__ = 'parsed_logs_final' # 새로운 테이블 이름 지정
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    client = Column(String(255))
    userid = Column(String(255))
    datetime = Column(String(255))
    method = Column(String(10))
    request = Column(Text) # request는 path가 될 수도 있지만, 그대로 사용
    status = Column(SmallInteger) # tinyint(4) 또는 smallint(6)에 해당하는 SmallInteger 사용
    size = Column(Integer) # int16이므로 Integer로 충분
    referer = Column(Text)
    user_agent = Column(Text)
    
# 5. ORM 모델에 정의된 테이블 생성
# 기존에 동일한 이름의 테이블이 있다면 먼저 삭제합니다.
# Base.metadata.drop_all(engine)
try:
    Base.metadata.create_all(engine)
    print("SQLAlchemy ORM을 사용하여 테이블이 성공적으로 생성되었습니다.")
except Exception as e:
    print(f"테이블 생성 실패: {e}")


# 6. Parquet 파일에서 데이터 불러오기
try:
    df = pd.read_parquet(PARQUET_FILE_PATH)
    print("DataFrame을 성공적으로 불러왔습니다.")
    
    # pandas to_sql의 dtype 인자 사용
    # 카테고리 타입은 문자열로 변환하여 저장합니다.
    df['method'] = df['method'].astype(str)
    df['status'] = df['status'].astype(str)
    df['size'] = df['size'].astype('int64') # MySQL INT 타입에 맞게 int64로 변환
    
except Exception as e:
    print(f"오류: {e}")
    exit()

# 7. DataFrame을 SQL 테이블에 저장
try:
    print(df.head())
    df.to_sql(
        name=ParsedLogs.__tablename__,
        con=engine,
        if_exists='append',
        index=False,
        chunksize=1000
    )
    print("데이터 저장 완료!")
except Exception as e:
    print(f"데이터 저장 실패: {e}")

MySQL 데이터베이스 연결 성공!
SQLAlchemy ORM을 사용하여 테이블이 성공적으로 생성되었습니다.
DataFrame을 성공적으로 불러왔습니다.
          client userid                    datetime method      request  \
0   52.79.152.10      -  16/Aug/2025:09:46:08 +0000   POST            /   
1  198.51.100.77      -  16/Aug/2025:06:15:55 +0000   POST       /login   
2    172.16.1.23    bob  17/Aug/2025:01:57:59 +0000    GET  /index.html   
3   52.79.152.10      -  18/Aug/2025:03:46:09 +0000    GET       /login   
4   192.168.0.10  alice  17/Aug/2025:22:44:40 +0000   POST            /   

  status  size                  referer  \
0    301   763    https://www.bing.com/   
1    401  2642    https://www.bing.com/   
2    503  2794    https://www.bing.com/   
3    503  3825  https://www.google.com/   
4    302  4090    https://www.bing.com/   

                                          user_agent  
0  Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like M...  
1  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  
2  Mozilla/5.0 (Macintosh; Intel Mac OS

## 테이블 삭제

In [11]:
from sqlalchemy import create_engine, text

# 데이터베이스 연결 엔진 생성
engine = create_engine(f'mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# 삭제할 테이블 이름
table_name = 'parsed_logs_final'

# SQL 쿼리 직접 실행
# 'IF EXISTS'를 추가하여 테이블이 존재하지 않더라도 오류가 발생하지 않게 합니다.
drop_query = text(f"DROP TABLE IF EXISTS {table_name}")

try:
    with engine.connect() as conn:
        conn.execute(drop_query)
        print(f"'{table_name}' 테이블이 성공적으로 삭제되었습니다.")
except Exception as e:
    print(f"'{table_name}' 테이블 삭제 실패: {e}")

'parsed_logs_final' 테이블이 성공적으로 삭제되었습니다.
